In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [5]:
df = pd.read_csv("blogtext.csv")
print(df.shape)
df.head()

(681284, 7)


,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


#### 2.	Preprocess rows of the “text” column (7.5 points)
#### a.	Remove unwanted characters
#### b.	Convert text to lowercase
#### c.	Remove unwanted spaces
#### d.	Remove stopwords

In [0]:
df['text'] = df['text'].apply(lambda x : x.lower())

In [0]:
from nltk.tokenize import word_tokenize, RegexpTokenizer
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
#removing stop words and unwanted characters/spaces 
for i in range(len(df['text'])):
    df['text'][i] = re.sub("\s\s+" , " ", df['text'][i])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyboardInterrupt: ignored

In [0]:
df['text'].head()

#### 3.	As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence (7.5 points)
#### a.	Label columns to merge: “gender”, “age”, “topic”, “sign”
#### b.	After completing the previous step, there should be only two columns in your data frame i.e. “text” and “labels”

In [10]:
df.columns[1:5]

Index(['gender', 'age', 'topic', 'sign'], dtype='object')

In [0]:
df['Label'] = df[df.columns[1:5]].apply(lambda x : ",".join(x.dropna().astype('str')),axis=1)

In [12]:
df['Label'].head(10)

0                   male,15,Student,Leo
1                   male,15,Student,Leo
2                   male,15,Student,Leo
3                   male,15,Student,Leo
4    male,33,InvestmentBanking,Aquarius
5    male,33,InvestmentBanking,Aquarius
6    male,33,InvestmentBanking,Aquarius
7    male,33,InvestmentBanking,Aquarius
8    male,33,InvestmentBanking,Aquarius
9    male,33,InvestmentBanking,Aquarius
Name: Label, dtype: object

In [13]:
df[:][['text','Label']].head()

,text,Label
0,"info has been found (+/- 100 pages, and 4.5 m...","male,15,Student,Leo"
1,these are the team members: drewes van der la...,"male,15,Student,Leo"
2,in het kader van kernfusie op aarde: maak je ...,"male,15,Student,Leo"
3,testing!!! testing!!!,"male,15,Student,Leo"
4,thanks to yahoo!'s toolbar i can now 'capture...,"male,33,InvestmentBanking,Aquarius"


In [0]:
df2 = df[:][['text','Label']]

#### 4.	Separate features and labels, and split the data into training and testing (5 points)
#### 5.	Vectorize the features (5 points)
#### a.	Create a Bag of Words using count vectorizer
#### i.	Use ngram_range=(1, 2)
#### ii.	Vectorize training and testing features
#### b.	Print the term-document matrix


In [0]:
X = df2['text']
Y = df2['Label']

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


In [0]:
# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=123)

In [0]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    #feature = nb.feature_count_
    #print("NB feature shape", feature)
   # print(nb.feature_count_.shape)
    y_train_pred = nb.predict(X_train_dtm)
    y_pred_class = nb.predict(X_test_dtm)
    print('Train Accuracy for NB : ', metrics.accuracy_score(y_train,y_train_pred))
    print('Test Accuracy for NB: ', metrics.accuracy_score(y_test, y_pred_class))
    logreg = LogisticRegression(C=1e9)
    logreg.fit(X_train_dtm, y_train)
    y_pred_class_LR = logreg.predict(X_test_dtm)
#print(metrics.accuracy_score(y_test, y_pred_class))
    y_train_LR = logreg.predict(X_train_dtm)
    print('Train Accuracy for LR: ',metrics.accuracy_score(y_train, y_train_LR))
    print('Test Accuracy for LR: ',metrics.accuracy_score(y_test, y_pred_class_LR))


In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vector = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vector)

Features:  13145454


In [1]:
X_train_dtm = vect.fit_transform(X_train)
print('Features: ', X_train_dtm.shape[1])
X_test_dtm = vect.transform(X_test)
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
    #feature = nb.feature_count_
    #print("NB feature shape", feature)
   # print(nb.feature_count_.shape)
y_train_pred = nb.predict(X_train_dtm)
y_pred_class = nb.predict(X_test_dtm)
print('Train Accuracy for NB : ', metrics.accuracy_score(y_train,y_train_pred))
print('Test Accuracy for NB: ', metrics.accuracy_score(y_test, y_pred_class))

NameError: ignored

In [0]:
# features names
features= vector.get_feature_names()
print(features[20:80])

#### 6.	Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label. Check below image for reference (5 points)

In [0]:
labels = {}
labels = y_train.apply(lambda x : pd.value_counts(x.split(","))).sum(axis = 0).to_dict()
labels


{'indUnk': 2461.0,
 'Sagittarius': 815.0,
 '24': 487.0,
 'female': 3090.0,
 '34': 425.0,
 '27': 785.0,
 'Education': 203.0,
 'Aquarius': 432.0,
 '36': 1290.0,
 'Aries': 3148.0,
 'Fashion': 1218.0,
 'male': 4410.0,
 '15': 455.0,
 'Libra': 367.0,
 'Student': 859.0,
 'Capricorn': 160.0,
 '16': 320.0,
 '35': 1723.0,
 'Technology': 1985.0,
 'Taurus': 600.0,
 'Pisces': 340.0,
 'Sports-Recreation': 57.0,
 '17': 891.0,
 'Leo': 230.0,
 'Scorpio': 738.0,
 '39': 54.0,
 'Virgo': 170.0,
 '26': 178.0,
 'Engineering': 99.0,
 '23': 184.0,
 '13': 34.0,
 'Cancer': 387.0,
 '14': 164.0,
 '25': 303.0,
 'Science': 54.0,
 'Communications-Media': 73.0,
 'Gemini': 113.0,
 'Marketing': 117.0,
 'Automotive': 12.0,
 'InvestmentBanking': 50.0,
 '33': 101.0,
 'Banking': 13.0,
 '42': 12.0,
 'Consulting': 17.0,
 '40': 1.0,
 'Internet': 88.0,
 'BusinessServices': 62.0,
 'Law': 6.0,
 'Arts': 35.0,
 '37': 25.0,
 '38': 33.0,
 '44': 3.0,
 'Museums-Libraries': 13.0,
 '43': 4.0,
 'Non-Profit': 59.0,
 'LawEnforcement-Securit

#### 7.	 Transform the labels - (7.5 points)
#### As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn
#### a.	Convert your train and test labels using MultiLabelBinarizer


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_mlb = mlb.fit_transform(y_train)
y_test_mlb = mlb.transform(y_test)

In [0]:
y_train_mlb[:5]

array([[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 1, 0, 1, 1, 0, 0, 0]])

In [0]:
y_test_mlb[:5]

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 0]])

#### Choose a classifier - (5 points)
#### In this task, we suggest using the One-vs-Rest approach, which is implemented in OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a basic classifier, use LogisticRegression. It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time because the number of classifiers to train is large.
#### a.	Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label
#### b.	As One-vs-Rest approach might not have been discussed in the sessions, we are providing you the code for that

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
model = LogisticRegression(solver = 'lbfgs',random_state= 123)

In [0]:
clf = OneVsRestClassifier(model)

In [0]:
feature_names = vector.get_feature_names()
feature_names

['aa',
 'aa amazing',
 'aa anger',
 'aa compared',
 'aa keeps',
 'aa nice',
 'aa sd',
 'aaa',
 'aaa come',
 'aaa coming',
 'aaa discount',
 'aaa rated',
 'aaa tow',
 'aaaa',
 'aaaa jet',
 'aaaaaaaaaaaah',
 'aaaaaaaaaaahhhhhhhhhhhhhhhhhhh',
 'aaaaaaaaaaahhhhhhhhhhhhhhhhhhh hw',
 'aaaaaaaaah',
 'aaaaaah',
 'aaaaaawwwwww',
 'aaaaaawwwwww gets',
 'aaaaack',
 'aaaaahhhh',
 'aaaaahhhh heath',
 'aaaah',
 'aaaah wisdom',
 'aaaahh',
 'aaaarrrgghhhh',
 'aaaarrrgghhhh slightly',
 'aaah',
 'aaah hafta',
 'aaahhhh',
 'aaahhhh diva',
 'aaahing',
 'aaahing mock',
 'aaarrggghhh',
 'aaarrggghhh plus',
 'aaarrggghhh thanks',
 'aaarrrggghhhhhhhhgggghhhhhh',
 'aaarrrggghhhhhhhhgggghhhhhh dropped',
 'aactually',
 'aactually hope',
 'aahed',
 'aahed god',
 'aaja',
 'aaja unity',
 'aal',
 'aal eliminate',
 'aal powerful',
 'aal representative',
 'aaldering',
 'aaldering urllink',
 'aar',
 'aar toy',
 'aarde',
 'aarde maak',
 'aargh',
 'aargh told',
 'aaron',
 'aaron broke',
 'aaron brown',
 'aaron burr',
 'a

####  9.	Fit the classifier, make predictions and get the accuracy (5 points)
####  a.	Print the following
#### i.	Accuracy score
#### ii.	F1 score
#### iii.	Average precision score
#### iv.	Average recall score
#### v.	Tip: Make sure you are familiar with all of them. How would you expect the things to work for the multi-label scenario? Read about micro/macro/weighted averaging


In [0]:
clf.fit(X_train_dtm,y_train_mlb)


/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))
/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/dsn/anaco

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=123,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
y_pred_clf = clf.predict(X_test_dtm)

In [0]:
metrics.accuracy_score(y_test_mlb,y_pred_clf)

0.2412

In [0]:
from sklearn.metrics import classification_report
classification_report(y_test_mlb, y_pred_clf)

/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00      2500\n           1       0.67      0.09      0.16        68\n           2       0.00      0.00      0.00         0\n           3       0.87      0.50      0.64       625\n           4       0.72      0.37      0.49       647\n           5       0.81      0.76      0.79      1298\n           6       0.84      0.36      0.50       363\n           7       0.75      0.51      0.61       827\n           8       0.88      0.44      0.59       596\n           9       0.77      0.30      0.43       571\n          10       1.00      0.15      0.27        13\n          11       1.00      0.04      0.08        25\n          12       0.81      0.73      0.77      1192\n          13       1.00      0.23      0.38        52\n          14       0.81      0.29      0.43       202\n          15       0.75      0.09      0.16        98\n          16       0.96      0.50      0.65       404\n       

In [0]:
print(classification_report(y_test_mlb, y_pred_clf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2500
           1       0.67      0.09      0.16        68
           2       0.00      0.00      0.00         0
           3       0.87      0.50      0.64       625
           4       0.72      0.37      0.49       647
           5       0.81      0.76      0.79      1298
           6       0.84      0.36      0.50       363
           7       0.75      0.51      0.61       827
           8       0.88      0.44      0.59       596
           9       0.77      0.30      0.43       571
          10       1.00      0.15      0.27        13
          11       1.00      0.04      0.08        25
          12       0.81      0.73      0.77      1192
          13       1.00      0.23      0.38        52
          14       0.81      0.29      0.43       202
          15       0.75      0.09      0.16        98
          16       0.96      0.50      0.65       404
          17       0.00    

In [0]:
metrics.average_precision_score(y_test_mlb, y_pred_clf,average='micro')

0.7786164795833284

In [0]:
#average='micro
metrics.recall_score(y_test_mlb, y_pred_clf,average='micro')

0.7765720580307967

In [0]:
#average='macro'
metrics.recall_score(y_test_mlb, y_pred_clf,average='macro')

/Users/dsn/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.4586565631405319

In [0]:
#average='weighted'
metrics.recall_score(y_test_mlb, y_pred_clf,average='weighted')

0.7765720580307967

In [0]:
metrics.multilabel_confusion_matrix(y_test_mlb, y_pred_clf,sample_weight=None,labels=None,samplewise=False)

array([[[   0,    0],
        [   0, 2500]],

       [[2429,    3],
        [  62,    6]],

       [[2500,    0],
        [   0,    0]],

       [[1827,   48],
        [ 310,  315]],

       [[1760,   93],
        [ 406,  241]],

       [[ 967,  235],
        [ 307,  991]],

       [[2113,   24],
        [ 233,  130]],

       [[1536,  137],
        [ 408,  419]],

       [[1869,   35],
        [ 331,  265]],

       [[1878,   51],
        [ 399,  172]],

       [[2487,    0],
        [  11,    2]],

       [[2475,    0],
        [  24,    1]],

       [[1096,  212],
        [ 316,  876]],

       [[2448,    0],
        [  40,   12]],

       [[2284,   14],
        [ 143,   59]],

       [[2399,    3],
        [  89,    9]],

       [[2087,    9],
        [ 203,  201]],

       [[2463,    0],
        [  37,    0]],

       [[2499,    1],
        [   0,    0]],

       [[2449,    1],
        [  40,   10]],

       [[2289,    9],
        [ 174,   28]],

       [[2430,    1],
        [  5

#### 10.	 Print true label and predicted label for any five examples (7.5 points)

In [0]:
y_pred_clf[100:105]

array([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 0, 0, 0, 0]])

In [0]:
y_test_mlb[100:105]

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0]])

In [0]:
labels.keys()

dict_keys(['indUnk', 'Sagittarius', '24', 'female', '34', '27', 'Education', 'Aquarius', '36', 'Aries', 'Fashion', 'male', '15', 'Libra', 'Student', 'Capricorn', '16', '35', 'Technology', 'Taurus', 'Pisces', 'Sports-Recreation', '17', 'Leo', 'Scorpio', '39', 'Virgo', '26', 'Engineering', '23', '13', 'Cancer', '14', '25', 'Science', 'Communications-Media', 'Gemini', 'Marketing', 'Automotive', 'InvestmentBanking', '33', 'Banking', '42', 'Consulting', '40', 'Internet', 'BusinessServices', 'Law', 'Arts', '37', '38', '44', 'Museums-Libraries', '43', 'Non-Profit', 'LawEnforcement-Security', '45', '41', 'HumanResources', 'Publishing', 'Religion', '46', 'Accounting'])

In [0]:
y_test_mlb.shape

(2500, 52)

In [0]:
labels_list = list(labels.keys())[:52]
labels_list

['indUnk',
 'Sagittarius',
 '24',
 'female',
 '34',
 '27',
 'Education',
 'Aquarius',
 '36',
 'Aries',
 'Fashion',
 'male',
 '15',
 'Libra',
 'Student',
 'Capricorn',
 '16',
 '35',
 'Technology',
 'Taurus',
 'Pisces',
 'Sports-Recreation',
 '17',
 'Leo',
 'Scorpio',
 '39',
 'Virgo',
 '26',
 'Engineering',
 '23',
 '13',
 'Cancer',
 '14',
 '25',
 'Science',
 'Communications-Media',
 'Gemini',
 'Marketing',
 'Automotive',
 'InvestmentBanking',
 '33',
 'Banking',
 '42',
 'Consulting',
 '40',
 'Internet',
 'BusinessServices',
 'Law',
 'Arts',
 '37',
 '38',
 '44']

In [0]:
#marking the labels 

import numpy as np
count=0
for i in range(5):
    pred_list = []
    true_list = []
    values = y_pred_clf[100+count:100+count+1]
    searchval = 1
    pred_indices = np.where(values == searchval)[1] 
    for i in pred_indices:
        pred_list.append(labels_list[i])
    values = y_test_mlb[100+count:100+count+1]
    print("predicted labels for {}th record : {} \n".format(100+count,pred_list))

    true_indices = np.where(values == searchval)[1]
    for k in true_indices:
        true_list.append(labels_list[k])
    print("true labels for {}th record : {}\n".format(100+count,true_list))
    print("---------------------------------------------------------------")
    count+=1





        

predicted labels for 100th record : ['indUnk', '27', '15', 'Virgo', '23', 'Cancer', '25', 'Communications-Media', 'Gemini', 'Marketing', 'InvestmentBanking', '33', 'Banking', '42', 'Internet', 'BusinessServices'] 

true labels for 100th record : ['indUnk', '27', 'Aquarius', '15', 'Virgo', '23', 'Cancer', '25', 'Communications-Media', 'Gemini', 'Marketing', 'InvestmentBanking', '33', 'Banking', '42', 'Internet', 'BusinessServices', '44']

---------------------------------------------------------------
predicted labels for 101th record : ['indUnk', '15', '23', '25', 'Communications-Media', 'Gemini', 'Marketing', 'InvestmentBanking', '33', 'Banking', '42', 'Internet', 'BusinessServices'] 

true labels for 101th record : ['indUnk', '34', 'Education', '39', '26', '23', '14', '25', 'Science', 'Communications-Media', 'Marketing', 'Automotive', 'InvestmentBanking', '33', 'Banking', 'Internet', 'BusinessServices', 'Law', 'Arts']

---------------------------------------------------------------
p